Note: Small sequences should work on CPUs only (say 8 cores)

Connect to Jupyter button should only appear after the ProteinMPNN NIM server starts

First verify that the NIM server is up, when it shows "ready" run the other cells

In [2]:
!curl localhost:$nimport/v1/health/ready

{"status":"ready"}

In [3]:
import requests
import os
import json
from pathlib import Path

In [4]:
def get_reduced_pdb():
    pdb = Path("1R42.pdb")
    if not pdb.exists():
        pdb.write_text(requests.get(f"https://files.rcsb.org/download/{pdb}").text)
    lines = filter(lambda line: line.startswith("ATOM"), pdb.read_text().split("\n"))
    return "\n".join(list(lines)[:200])


In [5]:
r = requests.post(
    url="http://localhost:"+os.environ['nimport']+"/biology/ipd/proteinmpnn/predict",
    json={
        "input_pdb": get_reduced_pdb(),
        "ca_only": False,
        "use_soluble_model": False,
        "sampling_temp": [0.1],
    },
)
print(r, "Saving to output.fa:\n", r.text[:200], "...")
Path("output.fa").write_text(json.loads(r.text)["mfasta"])


<Response [200]> Saving to output.fa:
 {"mfasta":">input, score=2.6678, global_score=2.6678, fixed_chains=[], designed_chains=['A'], model_name=v_48_002, git_hash=unknown, seed=102\nSTIEEQAKTFLDKFNHEAEDLFYQ\n>T=0.1, sample=1, score=1.2692, ...


254